### 1. Formulação do Problema de Programação Matemática

#### Variáveis de Decisão
- $z_{gt}$: número de geradores do tipo $g$ ligados e funcionando no período $t$.
- $w_{gt}$: número de geradores do tipo $g$ que foram ligados no período $t$ que não estavam ligados no período anterior $t-1$.
- $x_{gt}$: quantidade total de energia produzida no período $t$ pelos geradores do tipo $g$ que estiverem em funcionamento.

#### Função Objetivo
Minimizar o custo total de geração, que inclui o custo de setup (ligar os geradores), o custo fixo de operação no mínimo operacional e o custo variável de operar acima do mínimo:
$$ \text{Minimizar} \sum_{g \in G} \sum_{t \in T} \left( s_g w_{gt} + m_g c_g l_t z_{gt} + v_g l_t (x_{gt} - m_g z_{gt}) \right) $$

#### Restrições
1. Atendimento das faixas operacionais: $m_g z_{gt} \leq x_{gt} \leq M_g z_{gt}$ para todo $g \in G$ e $t \in T$.
2. Limites no número de geradores ligados: $z_{gt} \leq n_g$ para todo $g \in G$ e $t \in T$.
3. Atendimento da demanda: $\sum_{g \in G} x_{gt} \geq d_t$ para todo $t \in T$.
4. Flexibilidade operacional (reserva de 15%): $\sum_{g \in G} M_g z_{gt} \geq (1 + p) d_t$ para todo $t \in T$.
5. Acoplamento entre as variáveis $z_{gt}$ e $w_{gt}$: Restrições para garantir que $w_{gt}$ reflita corretamente o número de geradores ligados em cada período.

### Resolução do Modelo


In [20]:
from pyomo.environ import *
from pyomo.opt import SolverFactory

# Definição de conjuntos e parâmetros
T = [i for i in range(1, 6)]  # Períodos do dia
G = [1, 2, 3]  # Tipos de geradores
mg = {1: 850, 2: 1250, 3: 1500}
Mg = {1: 2000, 2: 1750, 3: 4000}
ng = {1: 12, 2: 10, 3: 5}
cg = {1: 1000, 2: 2600, 3: 3000}
vg = {1: 20, 2: 52, 3: 60}  # Diferença entre custo adicional e custo no mínimo
sg = {1: 2000, 2: 1000, 3: 500}
dt = {1: 15000, 2: 30000, 3: 25000, 4: 40000, 5: 27000}
lt = {1: 6, 2: 3, 3: 6, 4: 3, 5: 6}
p = 0.15

def make_model(model):        
    # Variáveis de decisão
    model.z = Var(G, T, within=NonNegativeIntegers)
    model.w = Var(G, T, within=NonNegativeIntegers)
    model.x = Var(G, T, within=NonNegativeReals)

    # Função objetivo
    model.cost = Objective(expr=sum(sg[g]*model.w[g,t] + cg[g]*mg[g]*lt[t]*model.z[g,t] + vg[g]*lt[t]*(model.x[g,t] - mg[g]*model.z[g,t]) for g in G for t in T), sense=minimize)
    return model

def make_constraints(model, operational_cost):
    # Restrições
    model.operational_limits = ConstraintList()
    for g in G:
        for t in T:
            model.operational_limits.add(mg[g]*model.z[g,t] <= model.x[g,t])
            model.operational_limits.add(model.x[g,t] <= Mg[g]*model.z[g,t])

    model.generator_limits = ConstraintList()
    for g in G:
        for t in T:
            model.generator_limits.add(model.z[g,t] <= ng[g])

    model.demand_satisfaction = ConstraintList()
    for t in T:
        model.demand_satisfaction.add(sum(model.x[g,t] for g in G) >= dt[t])

    if operational_cost:
        model.operational_flexibility = ConstraintList()
        for t in T:
            model.operational_flexibility.add(sum(Mg[g]*model.z[g,t] for g in G) >= (1 + p) * dt[t])
    
    return model


In [21]:
model_with = ConcreteModel()
model_with = make_model(model_with)
model_with = make_constraints(model_with, operational_cost=True)

solver = SolverFactory('gurobi')
solver.solve(model_with, tee=True)

total_with_operational_cost = value(model_with.cost)
print("Custo total de operação: ", total_with_operational_cost)
for g in G:
    for t in T:
        print(f"Geradores do tipo {g} ligados no período {t}: ", value(model_with.z[g,t]))
        print(f"Energia produzida pelo tipo {g} no período {t}: ", value(model_with.x[g,t]))

Set parameter CSManager to value "http://110.22.10.6:61080"
Set parameter CSAuthToken
Compute Server job ID: 6a478277-8911-4227-843c-3986ac5d5bdc
Capacity available on 'SRVAIOPT01' - connecting...
Established HTTP unencrypted connection
Read LP format model from file C:\Users\JEANLU~1.MOT\AppData\Local\Temp\tmpdc3xoett.pyomo.lp
Reading time = 0.28 seconds
x1: 55 rows, 45 columns, 105 nonzeros
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Gurobi Compute Server Worker version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 55 rows, 45 columns and 105 nonzeros
Model fingerprint: 0x9f48f430
Variable types: 15 continuous, 30 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [6e+01, 3e+07]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 5e+04]
Presolve removed 47 rows and 39 columns
Presolve time: 0.01s
Presolved: 8 rows, 6 columns, 18 nonzeros

In [22]:
model_without = ConcreteModel()
model_without = make_model(model_without)
model_without = make_constraints(model_without, operational_cost=False)

solver = SolverFactory('gurobi')
solver.solve(model_without, tee=True)

total_without_operational_cost = value(model_without.cost)
print("Custo total de operação: ", total_without_operational_cost)
for g in G:
    for t in T:
        print(f"Geradores do tipo {g} ligados no período {t}: ", value(model_without.z[g,t]))
        print(f"Energia produzida pelo tipo {g} no período {t}: ", value(model_without.x[g,t]))

Set parameter CSManager to value "http://110.22.10.6:61080"
Set parameter CSAuthToken
Compute Server job ID: 10d47e51-8df4-4219-98f9-40f8069742ca
Capacity available on 'SRVAIOPT01' - connecting...
Established HTTP unencrypted connection
Read LP format model from file C:\Users\JEANLU~1.MOT\AppData\Local\Temp\tmp6h4dm82s.pyomo.lp
Reading time = 0.31 seconds
x1: 50 rows, 45 columns, 90 nonzeros
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Gurobi Compute Server Worker version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 50 rows, 45 columns and 90 nonzeros
Model fingerprint: 0x3e09cc0a
Variable types: 15 continuous, 30 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [6e+01, 3e+07]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 4e+04]
Presolve removed 43 rows and 39 columns
Presolve time: 0.01s
Presolved: 7 rows, 6 columns, 15 nonzeros
V

In [23]:
economia = total_with_operational_cost - total_without_operational_cost
print("Economia ao remover a restrição de capacidade operacional adicional: ", economia)

Economia ao remover a restrição de capacidade operacional adicional:  106023000.0


A tarifa $T_t$ para um período $t$ pode ser calculada dividindo o custo total de operação pelo total de energia gerada (ou demanda atendida) nesse período. Vou considerar que a tarifa necessária para cobrir os custos seja proporcional ao custo total dividido pela demanda total. 

### Fórmula para calcular as tarifas:
$$T_t = \frac{\text{Custo Operacional}}{\text{Demanda Atendida}}$$

$$
\text{Custo Operacional de } t = \sum_{g \in G} \left( s_g \cdot w_{gt} + c_g \cdot m_g \cdot l_t \cdot z_{gt} + v_g \cdot l_t \cdot (x_{gt} - m_g \cdot z_{gt}) \right)
$$



In [24]:
def calcular_tarifas(model, dt, lt):    
    custos_operacionais = {t: sum(sg[g]*value(model.w[g,t]) + cg[g]*mg[g]*lt[t]*value(model.z[g,t]) + vg[g]*lt[t]*(value(model.x[g,t]) - mg[g]*value(model.z[g,t])) for g in G) for t in T}        
    tarifas = {t: custos_operacionais[t] / dt[t] for t in T}    
    return tarifas

tarifas_com = calcular_tarifas(model_with, dt, lt)
tarifas_sem = calcular_tarifas(model_without, dt, lt)

print("Tarifas por MW para cada hora do dia com garantia de capacidade adicional de 15%:")
for t in T:
    print(f"Período {t}: {tarifas_com[t]:.2f} USD/MW")

print("\nTarifas por MW para cada hora do dia sem garantia de capacidade adicional:")
for t in T:
    print(f"Período {t}: {tarifas_sem[t]:.2f} USD/MW")

Tarifas por MW para cada hora do dia com garantia de capacidade adicional de 15%:
Período 1: 3118.80 USD/MW
Período 2: 2406.60 USD/MW
Período 3: 4365.84 USD/MW
Período 4: 2987.10 USD/MW
Período 5: 4328.00 USD/MW

Tarifas por MW para cada hora do dia sem garantia de capacidade adicional:
Período 1: 2785.60 USD/MW
Período 2: 1890.30 USD/MW
Período 3: 3293.04 USD/MW
Período 4: 2180.70 USD/MW
Período 5: 3348.00 USD/MW
